In [1]:
import pymongo
import os
import time
import functools
import json
import threading

CONNECTION_CHAIN = os.getenv("CONNECTION_CHAIN")
base_size = int(os.getenv("INSERT_SIZE", 1))
iterations = int(os.getenv("INTERATIONS", 3))

def calcular_tiempo(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        inicio = time.time()
        resultado = func(*args, **kwargs)
        fin = time.time()
        tiempo = fin - inicio
        return resultado, tiempo
    return wrapper


def testear_conexion():
    try:
        cliente = pymongo.MongoClient(CONNECTION_CHAIN)
        lista_bases_de_datos = cliente.list_database_names()
        print("Conexión exitosa. Bases de datos disponibles:")
        for base_de_datos in lista_bases_de_datos:
            print(base_de_datos)
        cliente.close()
    except Exception as e:
        print("Error al conectar a MongoDB:", e)
testear_conexion()

DATABASES = ["compras", "ventas"]
ELEMENT_TYPE: list[str] = ["1_papel", "2_boligrafo", "3_lapiz", "4_goma", "5_regla"]

Conexión exitosa. Bases de datos disponibles:
admin
config
local
test


In [2]:
@calcular_tiempo
def insert_documents(size):
    cliente = pymongo.MongoClient(CONNECTION_CHAIN)
    for col_name in DATABASES:
        col = cliente.test[col_name]
        for element in ELEMENT_TYPE:
            for i in range(size):
                col.insert_one({"element": element, "amount": i})

@calcular_tiempo
def batch_insert_documents(size):
    cliente = pymongo.MongoClient(CONNECTION_CHAIN)
    for col_name in DATABASES:
        col = cliente.test[col_name]
        documents = []
        for element in ELEMENT_TYPE:
            for i in range(size):
                documents.append({"element": element, "amount": i})

        col.insert_many(documents)


def insert_documents_thread(size, col_name, element):
    cliente = pymongo.MongoClient(CONNECTION_CHAIN)
    col = cliente.test[col_name]
    for i in range(size):
        col.insert_one({"element": element, "amount": i})


@calcular_tiempo
def parallel_insert_documents(size):
    threads = []
    for col_name in DATABASES:
        for element in ELEMENT_TYPE:
            thread = threading.Thread(
                target=insert_documents_thread, args=(size, col_name, element)
            )
            threads.append(thread)
            thread.start()

    # Espera a que todos los threads terminen
    for thread in threads:
        thread.join()

In [3]:
@calcular_tiempo
def get_documents(database, element):
    cliente = pymongo.MongoClient(CONNECTION_CHAIN)
    col = cliente.test[database]
    result = list(col.find({"element": element}))
    return len(result)



@calcular_tiempo
def drop_elements(database, element):
    cliente = pymongo.MongoClient(CONNECTION_CHAIN)
    col = cliente.test[database]
    result = col.delete_many({"element": element})
    return result.deleted_count

In [23]:

def insert_data(DATABASES, ELEMENT_TYPE, base_size, i, iteration_result):
    insert_size = (2**i) * base_size * len(DATABASES) * len(ELEMENT_TYPE)
    insert_result = {"secuential":{},"batch":{},"parallel":{}}
    _, insert_spent_time = insert_documents(insert_size)
    insert_result["secuential"]["size"]= insert_size
    insert_result["secuential"]["time"]= insert_spent_time
    insert_result["secuential"]["rate"]= insert_size / insert_spent_time

    _, insert_spent_time = batch_insert_documents(insert_size)
    insert_result["batch"]["size"]= insert_size
    insert_result["batch"]["time"]= insert_spent_time
    insert_result["batch"]["rate"]= insert_size / insert_spent_time

    _, insert_spent_time = parallel_insert_documents(insert_size)
    insert_result["parallel"]["size"]= insert_size
    insert_result["parallel"]["time"]= insert_spent_time
    insert_result["parallel"]["rate"]= insert_size / insert_spent_time
    iteration_result["insert"] =insert_result


def query_data(DATABASES, ELEMENT_TYPE, iteration_result):
    iteration_result["find"] = {}
    for db in DATABASES:
        iteration_result["find"][db]={}
        find_size, find_spent_time = get_documents(db, ELEMENT_TYPE[0])
        iteration_result["find"][db]["time"] = find_spent_time
        iteration_result["find"][db]["size"] = find_size
        iteration_result["find"][db]["rate"] = find_size / find_spent_time


def drop_data(iteration_result):
    for i in DATABASES:
        for e in ELEMENT_TYPE:
            size, time = drop_elements(i,e)
            iteration_result[f"drop.{i}.{e}.size"] = size
            iteration_result[f"drop.{i}.{e}.time"] = time

In [24]:
result = {}
iterations = 2
attemts = 1
for i in range(iterations):
    result[f"I{i}"] = {}
    for j in range(attemts):
        iteration_result = {}
        docs = pymongo.MongoClient(CONNECTION_CHAIN).test.command("dbstats")["objects"]
        print(f"Starting iteration. Documents in db: {format(docs, ",").replace(",", ".")}")
        insert_data(DATABASES, ELEMENT_TYPE,  base_size, i, iteration_result)
        query_data(DATABASES, ELEMENT_TYPE,  iteration_result)
        drop_data(iteration_result)
        print(iteration_result)
        result[f"I{i}"][f"A{j}"] = iteration_result
    

filename = os.getenv("OUTPUT_FILENAME", "/output/output.json")

with open(filename, "w") as archivo:
    json.dump(result, archivo, indent=2)


print(f"\n\nTest complete. Output {filename} wrote")

Starting iteration. Documents in db: 402.236
{'insert': {'secuential': {'size': 10, 'time': 0.29404759407043457, 'rate': 34.0081000547301}, 'batch': {'size': 10, 'time': 0.021175622940063477, 'rate': 472.2411250098517}, 'parallel': {'size': 10, 'time': 0.19168615341186523, 'rate': 52.16860906230177}}, 'find': {'compras': {'time': 0.5743057727813721, 'size': 81950, 'rate': 142694.0209970637}, 'ventas': {'time': 0.011675834655761719, 'size': 30, 'rate': 2569.4094584660625}}, 'drop.compras.1_papel.size': 81950, 'drop.compras.1_papel.time': 1.2402064800262451, 'drop.compras.2_boligrafo.size': 81950, 'drop.compras.2_boligrafo.time': 1.2372605800628662, 'drop.compras.3_lapiz.size': 81950, 'drop.compras.3_lapiz.time': 1.2577471733093262, 'drop.compras.4_goma.size': 81950, 'drop.compras.4_goma.time': 1.1946630477905273, 'drop.compras.5_regla.size': 75239, 'drop.compras.5_regla.time': 1.1480038166046143, 'drop.ventas.1_papel.size': 30, 'drop.ventas.1_papel.time': 0.017700672149658203, 'drop.ven